In [1]:
import platform
import numpy as np
import ctypes
import time
import math
import time
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

In [2]:
!python --version

Python 3.10.13


## Code

In [3]:
input_array = np.random.uniform(-1, 1, 100).astype("float32") # data type have to be float32 for GPU
swift_result = swift_sigmoid_on_gpu(input_array)
swift_result

NameError: name 'swift_sigmoid_on_gpu' is not defined

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

python_result = sigmoid(input_array).astype("float32")

In [ ]:
# Returns a matrix whose elements are all 1, if the 2 vectors are same
cosine_similarity([python_result, python_result])

In [ ]:
cosine_similarity([swift_result, python_result])

## Benmark

In [ ]:
def f(x):
    approximate = 0
    for coeff in range(1, 10000, 2):
        approximate += (1/coeff)*np.sin(coeff*x)
        
    return approximate

In [ ]:
def differential(f, x):
    delta = 1e-4
    return (f(x+delta) - f(x-delta)) / 2*delta

### Numpy

In [ ]:
input_array = np.arange(-50, 50, 0.001).astype("float32") # have to be float32 for GPU
input_array.shape

In [ ]:
%%time
python_result = np.apply_along_axis(lambda x: differential(f, x), 0, input_array)

### Metal

In [ ]:
swift_fun = ctypes.CDLL("./PyMetalBridge/.build/release/libPyMetalBridge.dylib")

# decleare arguments type
swift_fun.swift_differential_on_gpu.argtypes = [
    ctypes.POINTER(ctypes.c_float), 
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int
]

def swift_differential_on_gpu(input_array):
    input_ptr = input_array.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
    output_mutable_ptr = (ctypes.c_float * len(input_array))()
    swift_fun.swift_differential_on_gpu(input_ptr, output_mutable_ptr, len(input_array))
    return np.array(output_mutable_ptr)

In [ ]:
%%time
swift_result = swift_differential_on_gpu(input_array)

In [ ]:
%%time
swift_result = swift_differential_on_gpu(input_array)

### Validate result

In [ ]:
swift_result.shape, python_result.shape

In [ ]:
cosine_similarity([python_result, python_result])

In [ ]:
cosine_similarity([swift_result, python_result.astype("float32")])

### Performance comparison between Numpy and Metal

In [ ]:
def measure(f):
    elapsed_logs = []
    for i in range(10):
        input_array = np.random.uniform(-50, 50, 100000).astype("float32")
        start = time.time()
        f(input_array)
        end = time.time()
        elapsed_logs.append(end - start)
        
    return elapsed_logs

In [ ]:
# np.apply_along_axis
python_results = measure(lambda input_array: np.apply_along_axis(lambda x: differential(f, x), 0, input_array))

In [ ]:
print("Python time elapsed (second)")
python_results

In [ ]:
# Metal
swift_results = measure(swift_differential_on_gpu)

In [ ]:
print("Metal time elapsed (second)")
swift_results

In [ ]:
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 4))

ax.plot(range(10), swift_results, '-o', label='Python+Metal')
ax.plot(range(10), python_results, '-o', label='Python + Numpy')
ax.set_title('Numpy and Metal performance comprison')
ax.set_xlabel('iteration')
ax.set_ylabel('time elapsed (second)')
ax.grid(True)
ax.legend()